In [1]:
import pandas as pd
import sqlite3
import json

orders = pd.read_csv("orders.csv")
with open("users.json") as f:
    users = pd.DataFrame(json.load(f))
conn = sqlite3.connect(":memory:")
with open("restaurants.sql") as f:
    conn.executescript(f.read())
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)

df = pd.merge(pd.merge(orders, users, on="user_id", how="left"), restaurants, on="restaurant_id", how="left")
df = df.rename(columns={"restaurant_name_x":"restaurant_order","restaurant_name_y":"restaurant_master"})
df.to_csv("final_food_delivery_dataset.csv", index=False)
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

gold_city_rev = df[df.membership=="Gold"].groupby("city")["total_amount"].sum().sort_values(ascending=False)
cuisine_aov = df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)
high_spenders = (df.groupby("user_id")["total_amount"].sum() > 1000).sum()
df["rating_range"] = pd.cut(df.rating, bins=[0,3.5,4.0,4.5,5.0], labels=["3.0–3.5","3.6–4.0","4.1–4.5","4.6–5.0"])
rating_rev = df.groupby("rating_range", observed=True)["total_amount"].sum().sort_values(ascending=False)
gold_city_aov = df[df.membership=="Gold"].groupby("city")["total_amount"].mean().sort_values(ascending=False)
cuisine_stats = df.groupby("cuisine").agg(total_revenue=("total_amount","sum"), restaurants=("restaurant_id","nunique")).sort_values(["restaurants","total_revenue"], ascending=[True,False])
gold_pct = round(df[df.membership=="Gold"].shape[0]/df.shape[0]*100)
rest_analysis = df.groupby("restaurant_order").agg(avg_order=("total_amount","mean"), orders=("order_id","count"))
top_low_freq = rest_analysis[rest_analysis.orders<20].sort_values("avg_order",ascending=False).head(1)
combo_rev = df.groupby(["membership","cuisine"])["total_amount"].sum().sort_values(ascending=False)
df["quarter"] = df.order_date.dt.quarter
quarter_rev = df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)

gold_orders = df[df.membership=="Gold"].shape[0]
hyd_revenue = round(df[df.city=="Hyderabad"]["total_amount"].sum())
distinct_users = df.user_id.nunique()
gold_avg_order = round(df[df.membership=="Gold"]["total_amount"].mean(),2)
high_rating_orders = df[df.rating>=4.5].shape[0]
top_gold_city = df[df.membership=="Gold"].groupby("city")["total_amount"].sum().idxmax()
gold_orders_top_city = df[(df.membership=="Gold") & (df.city==top_gold_city)].shape[0]

print("Highest revenue city (Gold):", gold_city_rev.head(1))
print("Cuisine with highest avg order:", cuisine_aov.head(1))
print("Users spending >1000:", high_spenders)
print("Rating range highest revenue:", rating_rev.head(1))
print("Gold members highest avg order city:", gold_city_aov.head(1))
print("Cuisine with fewest restaurants but high revenue:", cuisine_stats.head(1))
print("Percentage of Gold orders:", f"{gold_pct}%")
print("Highest avg order restaurant <20 orders:", top_low_freq)
print("Highest revenue membership+cuisine:", combo_rev.head(1))
print("Quarter with highest revenue:", quarter_rev.head(1))
print("Total Gold orders:", gold_orders)
print("Hyderabad revenue:", hyd_revenue)
print("Distinct users:", distinct_users)
print("Gold member avg order:", gold_avg_order)
print("Orders rating>=4.5:", high_rating_orders)
print("Orders in top Gold city:", gold_orders_top_city)


Highest revenue city (Gold): city
Chennai    1080909.79
Name: total_amount, dtype: float64
Cuisine with highest avg order: cuisine
Mexican    808.021344
Name: total_amount, dtype: float64
Users spending >1000: 2544
Rating range highest revenue: rating_range
4.6–5.0    2197030.75
Name: total_amount, dtype: float64
Gold members highest avg order city: city
Chennai    808.45908
Name: total_amount, dtype: float64
Cuisine with fewest restaurants but high revenue:          total_revenue  restaurants
cuisine                            
Chinese     1930504.65          120
Percentage of Gold orders: 50%
Highest avg order restaurant <20 orders:                             avg_order  orders
restaurant_order                             
Hotel Dhaba Multicuisine  1040.222308      13
Highest revenue membership+cuisine: membership  cuisine
Regular     Mexican    1072943.3
Name: total_amount, dtype: float64
Quarter with highest revenue: quarter
3    2037385.1
Name: total_amount, dtype: float64
Total G